In [1]:
import os

# ВСТАВЬ СЮДА СВОИ ЗНАЧЕНИЯ
os.environ["TELEGRAM_BOT_TOKEN"] = "8250825015:AAE9nIh5RLmbjNFl2yS0m3sBUKhfi3VJXd8"
os.environ["GIGACHAT_CREDENTIALS"] = "MmI2ZGMxMzktNDRmNC00ODYyLWJhYzQtZWY1YWFhY2RiYjAwOjI2OGEwNGVjLWYwNTQtNDUwMC05MjJhLTYwOTk3YWMzYzJhNA=="

In [2]:
!pip install -qU langchain-text-splitters
!pip install -qU langchain-community faiss-cpu
!pip install -qU sentence-transformers




In [3]:
!pip install \
  python-telegram-bot==20.8 \
  langchain-gigachat==0.3.13a2 \
  "langgraph>=1.0.4" \
  "langchain>=1.0.0" \
  python-dotenv==1.0.1 \
  openpyxl==3.1.5 \
  nest_asyncio==1.6.0 \
  pandas==2.2.3

In [5]:
import os
import asyncio
import logging
import re
from io import BytesIO
from typing import Sequence

import pandas as pd
from typing_extensions import Annotated, TypedDict

from telegram import Update, InputFile
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

from langchain_gigachat.chat_models import GigaChat
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.documents import Document

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# --------------------------------------------------------------------
# 0. Базовая настройка логирования
# --------------------------------------------------------------------
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)

# --------------------------------------------------------------------
# 1. Настройки: читаем токены из окружения
# --------------------------------------------------------------------
TELEGRAM_BOT_TOKEN = os.environ.get("TELEGRAM_BOT_TOKEN")
GIGACHAT_CREDENTIALS = os.environ.get("GIGACHAT_CREDENTIALS")

if not TELEGRAM_BOT_TOKEN:
    raise RuntimeError("Не задан TELEGRAM_BOT_TOKEN в переменных окружения")

if not GIGACHAT_CREDENTIALS:
    raise RuntimeError("Не задан GIGACHAT_CREDENTIALS в переменных окружения")

# --------------------------------------------------------------------
# 2. IT-ресурсы для мини-RAG
# --------------------------------------------------------------------
IT_RESOURCES = [
    {
        "topic": "python_beginner",
        "title": "MIT OCW: Introduction to Computer Science and Programming in Python",
        "url": "https://ocw.mit.edu/courses/6-0001-introduction-to-computer-science-and-programming-in-python-fall-2016/",
        "lang": "EN",
        "level": "beginner",
        "description": "Вводный курс по основам программирования и информатики на Python."
    },
    {
        "topic": "cs_core",
        "title": "MIT OCW: Introductory Programming Collection",
        "url": "https://ocw.mit.edu/collections/introductory-programming/",
        "lang": "EN",
        "level": "intermediate",
        "description": "Подборка базовых курсов по программированию и компьютерным наукам."
    },
    {
        "topic": "cs_many_courses",
        "title": "500+ Free Computer Science Courses",
        "url": "https://dev.to/manocormen/500-free-computer-science-online-courses-from-the-world-s-top-cs-universities-3pg5",
        "lang": "EN",
        "level": "mixed",
        "description": "Большая коллекция бесплатных онлайн-курсов по информатике от ведущих вузов."
    },
    {
        "topic": "cs_1200_courses",
        "title": "1200 Free Computer Science Courses",
        "url": "https://www.freecodecamp.org/news/free-courses-top-cs-universities/",
        "lang": "EN",
        "level": "mixed",
        "description": "Список бесплатных CS-курсов от топовых университетов."
    },
    {
        "topic": "python_ru",
        "title": "Курс: Программирование для всех (Python, RU-интерфейс)",
        "url": "https://www.coursera.org/learn/python-ru",
        "lang": "RU",
        "level": "beginner",
        "description": "Начальный курс по Python с русскоязычным интерфейсом."
    },
    {
        "topic": "it_ru_platform",
        "title": "Stepik – IT-курсы",
        "url": "https://stepik.org/catalog/search?tag=programming",
        "lang": "RU",
        "level": "mixed",
        "description": "Каталог курсов по программированию и другим IT-направлениям."
    },
    {
    "topic": "cs_ossu_path",
    "title": "OSSU: Computer Science Curriculum",
    "url": "https://github.com/ossu/computer-science",
    "lang": "EN",
    "level": "mixed",
    "description": "Полная дорожная карта самообучения компьютерным наукам из бесплатных онлайн‑курсов ведущих университетов."
  },
  {
      "topic": "web_fullstack_en",
      "title": "freeCodeCamp: Full Stack Web Development",
      "url": "https://www.freecodecamp.org",
     "lang": "EN",
      "level": "beginner",
      "description": "Бесплатная практическая программа по веб‑разработке и смежным направлениям с интерактивными заданиями и проектами."
  },
  {
      "topic": "cs_university_free_en",
      "title": "Free University CS Courses (Open Culture / freeCodeCamp list)",
      "url": "https://www.freecodecamp.org/news/free-courses-top-cs-universities/",
      "lang": "EN",
      "level": "mixed",
      "description": "Подборка сотен бесплатных курсов по информатике от топ‑университетов мира."
  },
  {
     "topic": "cs_harvard_free",
     "title": "Harvard Online: Free Courses (CS & Programming)",
      "url": "https://pll.harvard.edu/catalog/free",
     "lang": "EN",
      "level": "mixed",
      "description": "Каталог бесплатных онлайн‑курсов Гарварда, включая направления по программированию и компьютерным наукам."
  },
  {
      "topic": "cs_openlearn",
      "title": "OpenLearn: Digital & Computing",
      "url": "https://www.open.edu/openlearn/digital/free-courses",
      "lang": "EN",
      "level": "beginner",
      "description": "Бесплатные интро‑курсы по компьютерам, ИТ и программированию от Open University."
  },
  {
    "topic": "ru_yandex_practicum_free",
    "title": "Яндекс Практикум: бесплатные курсы по программированию",
    "url": "https://practicum.yandex.ru/catalog/programming/free/",
    "lang": "RU",
    "level": "beginner",
    "description": "Подборка бесплатных вводных курсов по программированию от Яндекс Практикума."
  },
  {
      "topic": "ru_hexlet_free",
      "title": "Hexlet: бесплатные курсы по IT‑профессиям",
      "url": "https://ru.hexlet.io/courses_free",
      "lang": "RU",
      "level": "beginner",
      "description": "Бесплатные стартовые курсы по программированию и смежным IT‑направлениям с упором на практику."
  },
  {
      "topic": "ru_netology_free",
      "title": "Нетология: бесплатные материалы по разработке",
      "url": "https://netology.ru/free/development",
      "lang": "RU",
      "level": "beginner",
      "description": "Бесплатные вводные курсы, вебинары и материалы по разработке и смежным digital‑направлениям."
  },
  {
      "topic": "ru_code_basics",
      "title": "CodeBasics: основы программирования",
      "url": "https://code-basics.com/ru",
      "lang": "RU",
      "level": "beginner",
      "description": "Интерактивные упражнения по основам разных языков программирования: Python, JavaScript, Go и др."
  }

]

# --------------------------------------------------------------------
# 3. Парсинг Markdown-таблицы и конвертация в Excel
# --------------------------------------------------------------------
def parse_markdown_table_to_df(text: str) -> pd.DataFrame:
    lines = text.strip().split('\n')
    table_lines = []
    table_started = False

    for line in lines:
        stripped = line.strip()
        if not table_started:
            if stripped.startswith('|') and len(stripped) > 3:
                table_started = True
            else:
                continue
        if stripped.startswith('|') and stripped.endswith('|'):
            cells = [cell.strip() for cell in stripped.split('|')[1:-1]]
            if any(cell for cell in cells):
                table_lines.append(cells)
        elif table_started:
            break

    if len(table_lines) < 2:
        raise ValueError("Недостаточно строк для таблицы (требуется заголовок + ≥1 строка)")

    headers = table_lines[0]
    data_rows = [
        row for row in table_lines[1:]
        if not all(re.fullmatch(r'-+', cell) for cell in row)
    ]

    if not data_rows:
        raise ValueError("Нет данных в таблице (только заголовок и разделитель)")

    return pd.DataFrame(data_rows, columns=headers)


def extract_links(text: str) -> list:
    LINK_PATTERN = r'\[([^\]]+)\]\(\s*([^)]*?)\s*\)'
    matches = re.findall(LINK_PATTERN, text)
    links = [{"title": title.strip(), "url": url.strip()} for title, url in matches]

    text_without_links = re.sub(LINK_PATTERN, '', text).strip()
    if text_without_links and not re.match(r'^[\s,]*$', text_without_links):
        return [{"context": text_without_links, "links": links}] if links else [{"raw": text.strip()}]

    return links if links else [{"raw": text.strip()}]


def df_to_excel_bytes(df: pd.DataFrame) -> BytesIO:
    try:
        df_out = df.copy()

        # Служебная обработка, как и раньше
        if 'Рекомендуемые ресурсы' in df_out.columns:
            df_out['resources'] = df_out['Рекомендуемые ресурсы'].apply(extract_links)

        if 'Продолжительность' in df_out.columns:
            def parse_duration(s):
                if not s:
                    return 1
                m = re.search(r'(\d+)', s)
                return int(m.group(1)) if m else 1

            df_out['duration_weeks'] = df_out['Продолжительность'].apply(parse_duration)

        if 'Рекомендуемая дата завершения' in df_out.columns:
            def parse_week_range(s):
                if not s:
                    return {"start": 1, "end": 1}
                m = re.search(r'(\d+)(?:\s*-\s*(\d+))?', s)
                if m:
                    start = int(m.group(1))
                    end = int(m.group(2)) if m.group(2) else start
                    return {"start": start, "end": end}
                return {"raw": s}

            df_out['completion'] = df_out['Рекомендуемая дата завершения'].apply(parse_week_range)

        # 👉 Оставляем для пользователя только видимые столбцы
        hidden_cols = ['resources', 'duration_weeks', 'completion']
        visible_cols = [c for c in df_out.columns if c not in hidden_cols]

        from openpyxl.utils import get_column_letter

        output = BytesIO()
        with pd.ExcelWriter(output, engine='openpyxl') as writer:
            # Пишем только видимые пользователю колонки
            df_out[visible_cols].to_excel(writer, sheet_name='План', index=False)

            worksheet = writer.sheets['План']
            for i, col in enumerate(visible_cols, start=1):
                max_len = max(df_out[col].astype(str).map(len).max(), len(col)) + 2
                worksheet.column_dimensions[get_column_letter(i)].width = min(max_len, 50)

        output.seek(0)
        return output

    except Exception:
        logging.exception("Ошибка в df_to_excel_bytes")
        raise


# --------------------------------------------------------------------
# 4. Модель GigaChat (чат) и эмбеддинги
# --------------------------------------------------------------------
model = GigaChat(
    credentials=GIGACHAT_CREDENTIALS,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Max",
    verify_ssl_certs=False,
)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
)

# --------------------------------------------------------------------
# 5. Построение векторного хранилища по IT_RESOURCES
# --------------------------------------------------------------------
docs = []
for res in IT_RESOURCES:
    content = (
        f"{res['title']}. {res['description']} "
        f"Язык: {res['lang']}. Уровень: {res['level']}."
    )
    docs.append(Document(page_content=content, metadata=res))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)
doc_splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(doc_splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# --------------------------------------------------------------------
# 6. Системный промпт для RAG-ассистента по обучению IT
# --------------------------------------------------------------------
RAG_SYSTEM_PROMPT = (
    "Ты — интеллектуальный помощник по обучению в сфере IT. "
    "Твоя основная задача — составлять персонализированные учебные планы в виде "
    "структурированной Markdown-таблицы на основе требований пользователя, "
    "ИСПОЛЬЗУЯ предоставленный контекст ресурсов (курсы, платформы, подборки).\n\n"
    "Контекст доступных IT-ресурсов:\n{context}\n\n"
    "Требования к таблице:\n"
    "1. Таблица ДОЛЖНА начинаться со строки '### Учебный план' и содержать РОВНО 7 колонок в этом порядке:\n"
    "   № | Тема/Модуль | Цель изучения | Рекомендуемые ресурсы | Формат занятий | Продолжительность | Рекомендуемая дата завершения\n"
    "2. В колонке 'Рекомендуемые ресурсы' используй ТОЛЬКО формат ссылок Markdown: [Название](URL).\n"
    "   Если ресурсов несколько — перечисли через запятую: [A](url1), [B](url2).\n"
    "3. В колонке 'Продолжительность' указывай ТОЛЬКО число и единицу: '1 неделя', '2 недели', '3 недели'.\n"
    "4. В колонке 'Рекомендуемая дата завершения' используй формат: 'неделя 1', 'недели 2-3', 'недели 8-9'.\n"
    "5. Перед тем как строить план, при необходимости задай уточняющие вопросы пользователю.\n"
    "6. Подбирая ресурсы, в первую очередь используй те, что есть в контексте."
)

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="messages"),
])

# --------------------------------------------------------------------
# 7. Состояние для LangGraph и узлы RAG-графа
# --------------------------------------------------------------------
class RAGState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    context: str

def retrieve_node(state: RAGState) -> dict:
    last_human = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            last_human = m
            break
    query = last_human.content if last_human else ""
    docs = retriever.invoke(query)
    context_text = "\n\n".join(d.page_content for d in docs)
    return {"context": context_text}

def call_rag_model(state: RAGState) -> dict:
    chain = rag_prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}

# --------------------------------------------------------------------
# 8. Сборка RAG-графа с памятью
# --------------------------------------------------------------------
workflow = StateGraph(state_schema=RAGState)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("model", call_rag_model)

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "model")
workflow.add_edge("model", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# --------------------------------------------------------------------
# 9. Telegram-обработчики
# --------------------------------------------------------------------
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Меня зовут Lumen. Я помогу тебе составить IT-учебный план 📚\n"
        "Напиши, что хочешь изучить и за какое время."
    )

async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = str(update.effective_user.id)
    config = {"configurable": {"thread_id": user_id}}
    await app.aupdate_state(config, {"messages": []})
    await update.message.reply_text("Память очищена!")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = str(update.effective_user.id)
    text = (update.message.text or "").strip()
    config = {"configurable": {"thread_id": user_id}}

    thinking_msg = await update.message.reply_text("Пожалуйста подождите... ⏳")

    try:
        # Один вызов RAG-графа
        output = await asyncio.wait_for(
            app.ainvoke(
                {"messages": [HumanMessage(content=text)], "context": ""},
                config,
            ),
            timeout=90.0,
        )

        plan_text = output["messages"][-1].content

        # Пытаемся сделать Excel из ответа
        try:
            df = parse_markdown_table_to_df(plan_text)
            excel_bytes = df_to_excel_bytes(df)
            await thinking_msg.delete()
            await update.message.reply_document(
                document=InputFile(excel_bytes, filename="plan.xlsx"),
                caption="Ваш учебный план в Excel",
            )
        except Exception:
            logging.exception("Ошибка конвертации в Excel")
            await thinking_msg.edit_text(plan_text[:4000])

    except Exception as e:
        import traceback
        err_text = "".join(traceback.format_exception_only(type(e), e))
        logging.exception(f"Критическая ошибка при обработке запроса от {user_id}")
        try:
            await thinking_msg.edit_text("Внутренняя ошибка:\n" + err_text[:3500])
        except Exception:
            await update.message.reply_text("Внутренняя ошибка:\n" + err_text[:3500])

# --------------------------------------------------------------------
# 10. Запуск бота (адаптирован под Colab / Jupyter)
# --------------------------------------------------------------------
async def main():
    application = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("clear", clear))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("✅ Бот запускается... Напишите ему в Telegram.")
    await application.initialize()
    await application.start()
    await application.updater.start_polling()
    print("✅ Бот запущен и слушает сообщения")

# Запуск с учётом Colab / Jupyter
loop = asyncio.get_event_loop()
if loop.is_running():
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.create_task(main())
else:
    asyncio.run(main())
